In [1]:
import pandas as pd

import sys
sys.path.append('../')
from utils import *

pd.set_option('display.max_columns', None)

In [2]:
DATA_PATH = '../../data/'
WYSCOUT_PATH = DATA_PATH + 'wyscout/'
SKILLCORNER_PATH = DATA_PATH + 'skillcorner/'
SKILLCORNER_RAW_PATH = DATA_PATH + 'skillcorner_raw/'
XT_PLOT_PATH = DATA_PATH + 'xt_plot_worldcup.csv'
MATCH_IDS_PATH = DATA_PATH + 'matchids.csv'
WYSCOUT_TO_SKILLCORNER = DATA_PATH + 'wyscout2skillcorner.csv'

## Load Data

In [3]:
match_info = pd.read_csv(MATCH_IDS_PATH)
match_info = match_info.dropna(subset=['wyscout', 'skillcorner'])
match_info['skillcorner'] = match_info['skillcorner'].astype(int)

In [4]:
match_info.head(5)

,wyscout,skillcorner,date,home,result,away
8,5414111,952209,2023-03-24,France,4-0,Netherlands
10,5414113,726849,2023-03-24,Czech Republic,3-1,Poland
12,5414115,722109,2023-03-24,Sweden,0-3,Belgium
25,5414127,726848,2023-03-26,England,2-0,Ukraine
33,5414135,952208,2023-03-27,Republic of Ireland,0-1,France


In [5]:
wyscout_id = match_info['wyscout'].values[0]
skillcorner_id = match_info['skillcorner'].values[0]

### wyscout

In [6]:
wyscout_data = wyscout_to_df(WYSCOUT_PATH + str(wyscout_id) + ".json")
with pd.option_context('display.max_columns', None):
    display(wyscout_data.head(3))

,id,matchId,matchPeriod,minute,second,matchTimestamp,videoTimestamp,relatedEventId,shot,groundDuel,aerialDuel,infraction,carry,type.primary,type.secondary,location.x,location.y,team.id,team.name,team.formation,opponentTeam.id,opponentTeam.name,opponentTeam.formation,player.id,player.name,player.position,pass.accurate,pass.angle,pass.height,pass.length,pass.recipient.id,pass.recipient.name,pass.recipient.position,pass.endLocation.x,pass.endLocation.y,possession.id,possession.duration,possession.types,possession.eventsNumber,possession.eventIndex,possession.startLocation.x,possession.startLocation.y,possession.endLocation.x,possession.endLocation.y,possession.team.id,possession.team.name,possession.team.formation,possession.attack,pass,carry.progression,carry.endLocation.x,carry.endLocation.y,groundDuel.opponent.id,groundDuel.opponent.name,groundDuel.opponent.position,groundDuel.duelType,groundDuel.keptPossession,groundDuel.progressedWithBall,groundDuel.stoppedProgress,groundDuel.recoveredPossession,groundDuel.takeOn,groundDuel.side,groundDuel.relatedDuelId,possession.attack.withShot,possession.attack.withShotOnGoal,possession.attack.withGoal,possession.attack.flank,possession.attack.xg,shot.bodyPart,shot.isGoal,shot.onTarget,shot.goalZone,shot.xg,shot.postShotXg,shot.goalkeeperActionId,shot.goalkeeper.id,shot.goalkeeper.name,possession,aerialDuel.opponent.id,aerialDuel.opponent.name,aerialDuel.opponent.position,aerialDuel.opponent.height,aerialDuel.firstTouch,aerialDuel.height,aerialDuel.relatedDuelId,shot.goalkeeper,infraction.yellowCard,infraction.redCard,infraction.type,infraction.opponent.id,infraction.opponent.name,infraction.opponent.position,infraction.opponent
0,1587492614,5414111,1H,0,1,00:00:01.912,3.912638,1.587493e+09,NaN,NaN,NaN,NaN,NaN,pass,"[lateral_pass, short_or_medium_pass]",51,50,664,Netherlands,4-2-3-1,4418,France,4-3-3,118,M. Depay,CF,True,114.0,None,24.0,625.0,M. de Roon,RDMF,42.0,83.0,1.587493e+09,110.275779,[],43.0,0.0,51.0,50.0,35.0,67.0,664.0,Netherlands,4-2-3-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1587492615,5414111,1H,0,5,00:00:05.782,7.782226,1.587493e+09,NaN,NaN,NaN,NaN,NaN,pass,"[lateral_pass, short_or_medium_pass]",42,83,664,Netherlands,4-2-3-1,4418,France,4-3-3,625,M. de Roon,RDMF,True,-133.0,None,21.0,476361.0,L. Geertruida,RCB,29.0,60.0,1.587493e+09,110.275779,[],43.0,1.0,51.0,50.0,35.0,67.0,664.0,Netherlands,4-2-3-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1587492616,5414111,1H,0,8,00:00:08.951,10.951834,1.587493e+09,NaN,NaN,NaN,NaN,NaN,pass,"[lateral_pass, short_or_medium_pass]",29,60,664,Netherlands,4-2-3-1,4418,France,4-3-3,476361,L. Geertruida,RCB,True,-100.0,None,22.0,370.0,V. van Dijk,LCB,25.0,28.0,1.587493e+09,110.275779,[],43.0,2.0,51.0,50.0,35.0,67.0,664.0,Netherlands,4-2-3-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### skillcorner

In [7]:
metadata = pd.read_csv(SKILLCORNER_PATH + str(skillcorner_id) + "_metadata.csv")
tracking_df = pd.read_csv(SKILLCORNER_PATH + str(skillcorner_id) + "_tracking.csv")
lineup_df = pd.read_csv(SKILLCORNER_PATH + str(skillcorner_id) + "_lineup.csv")
play_direction_df = pd.read_csv(SKILLCORNER_PATH + str(skillcorner_id) + "_play_direction.csv")

In [8]:
print("Metadata")
display(metadata.head(3))
print("Tracking")
display(tracking_df.head(3))
print("Lineup")
display(lineup_df.head(3))
print("Play Direction")
display(play_direction_df)


Metadata


,match_id,match_date,competition,season,home_team,away_team,home_score,away_score,home_team_jersey_color,away_team_jersey_color,home_team_number_color,away_team_number_color,home_team_coach,away_team_coach,pitch_name,pitch_length,pitch_width,provider,fps
0,952209,03/24/2023 19:45,European Championship Qualifiers,2023/2024,France,Netherland,4,0,#fafafa,#ff824c,#000000,#000000,Didier Deschamps,Frank De Boer,Stade de France,105.0,68.0,SkillCorner,10


Tracking


,match_id,half,frame_id,timestamp,object_id,x,y,z,extrapolated
0,952209,1,20,0,-1,0.12,-0.19,0.12,False
1,952209,1,20,0,7604,-39.44,-0.21,0.00,True
2,952209,1,20,0,16320,-16.05,-7.18,0.00,False


Lineup


,match_id,team_name,player_id,player_first_name,player_last_name,player_shirt_number,player_position,player_birthdate,start_time,end_time,yellow_card,red_card,injured,goal,own_goal
0,952209,Netherland,14397,Tyrell,Malacia,16,Right Wing Back,1999-08-17,01:26:59,NaN,0,0,False,0,0
1,952209,Netherland,12654,Donyell,Malen,7,Right Winger,1999-01-19,01:06:22,NaN,0,0,False,0,0
2,952209,Netherland,2548,Davy,Klaassen,14,Attacking Midfield,1993-02-21,01:06:22,NaN,0,0,False,0,0


Play Direction


,match_id,team_name,half,play_direction
0,952209,France,1,BOTTOM_TO_TOP
1,952209,Netherland,1,TOP_TO_BOTTOM
2,952209,France,2,TOP_TO_BOTTOM
3,952209,Netherland,2,BOTTOM_TO_TOP


# Pass Events

In [9]:
passes_df = wyscout_data[wyscout_data['type.primary'] == 'pass']

# Filter out inaccurate passes
accurate_only = True
if accurate_only:
    passes_df = passes_df[passes_df['pass.accurate'] == True]
    
# Filter out passes with invalid recipient id
passes_df = passes_df[passes_df['pass.recipient.id'] != 0]

print(passes_df.shape)

(894, 93)


In [10]:
passes = passes_df.loc[:,['matchTimestamp', 'matchPeriod', 'player.id', 'pass.recipient.id', 'location.x', 'location.y', 'pass.endLocation.x', 'pass.endLocation.y']]
passes.head()

,matchTimestamp,matchPeriod,player.id,pass.recipient.id,location.x,location.y,pass.endLocation.x,pass.endLocation.y
0,00:00:01.912,1H,118,625.0,51,50,42.0,83.0
1,00:00:05.782,1H,625,476361.0,42,83,29.0,60.0
2,00:00:08.951,1H,476361,370.0,29,60,25.0,28.0
3,00:00:13.151,1H,370,476361.0,25,28,25.0,51.0
5,00:00:17.930,1H,476361,370.0,32,50,30.0,17.0


In [11]:
passes['matchPeriod'] = passes['matchPeriod'].apply(lambda x: int(x.split('H')[0]))
passes['pass.recipient.id'] = passes['pass.recipient.id'].astype(int)
passes['pass.endLocation.x'] = passes['pass.endLocation.x'].astype(int)
passes['pass.endLocation.y'] = passes['pass.endLocation.y'].astype(int)

In [12]:
passes.head()

,matchTimestamp,matchPeriod,player.id,pass.recipient.id,location.x,location.y,pass.endLocation.x,pass.endLocation.y
0,00:00:01.912,1,118,625,51,50,42,83
1,00:00:05.782,1,625,476361,42,83,29,60
2,00:00:08.951,1,476361,370,29,60,25,28
3,00:00:13.151,1,370,476361,25,28,25,51
5,00:00:17.930,1,476361,370,32,50,30,17


# Map Wyscout IDs to SkillCorner IDs

In [13]:
wyscout2skillcorner = pd.read_csv(WYSCOUT_TO_SKILLCORNER).drop(columns='id')

In [14]:
passes.rename(columns={'player.id': 'player.id.wyscout', 'pass.recipient.id': 'pass.recipient.id.wyscout'}, inplace=True)

In [15]:
passes = passes.merge(wyscout2skillcorner[['player_id_sk', 'player_id_wy']], left_on='player.id.wyscout', right_on='player_id_wy', how='left')
passes.drop(columns=['player_id_wy'], inplace=True)
passes.rename(columns={'player_id_sk': 'player.id.skillcorner'}, inplace=True)

passes = passes.merge(wyscout2skillcorner[['player_id_sk', 'player_id_wy']], left_on='pass.recipient.id.wyscout', right_on='player_id_wy', how='left')
passes.drop(columns=['player_id_wy'], inplace=True)
passes.rename(columns={'player_id_sk': 'pass.recipient.id.skillcorner'}, inplace=True)

In [16]:
cols = ['matchTimestamp', 'matchPeriod', 'player.id.wyscout', 'player.id.skillcorner', 'pass.recipient.id.wyscout', 'pass.recipient.id.skillcorner', 'location.x', 'location.y', 'pass.endLocation.x', 'pass.endLocation.y']
passes = passes.loc[:,cols]

In [17]:
passes.head()

,matchTimestamp,matchPeriod,player.id.wyscout,player.id.skillcorner,pass.recipient.id.wyscout,pass.recipient.id.skillcorner,location.x,location.y,pass.endLocation.x,pass.endLocation.y
0,00:00:01.912,1,118,7387,625,7017,51,50,42,83
1,00:00:05.782,1,625,7017,476361,14407,42,83,29,60
2,00:00:08.951,1,476361,14407,370,10738,29,60,25,28
3,00:00:13.151,1,370,10738,476361,14407,25,28,25,51
4,00:00:17.930,1,476361,14407,370,10738,32,50,30,17


# Lineup Information

# Play Direction

In [18]:
play_direction_dict = play_direction_df.drop(columns='match_id').set_index(['team_name', 'half']).to_dict()['play_direction']
play_direction_dict

{('France', 1): 'BOTTOM_TO_TOP',
 ('Netherland', 1): 'TOP_TO_BOTTOM',
 ('France', 2): 'TOP_TO_BOTTOM',
 ('Netherland', 2): 'BOTTOM_TO_TOP'}

In [19]:
player_team_dict = lineup_df[['player_id', 'team_name']].set_index('player_id').to_dict()['team_name']

In [20]:
def get_play_direction(row):
    team_name = player_team_dict[row['player.id.skillcorner']]
    return play_direction_dict[(team_name, row['matchPeriod'])]

In [21]:
passes['play_direction'] = passes.apply(get_play_direction, axis=1)

# Player Locations

In [22]:
columns_to_drop = ['frame_id', 'extrapolated']
existing_columns_to_drop = [col for col in columns_to_drop if col in tracking_df.columns]
tracking_df.drop(columns=existing_columns_to_drop, inplace=True)

In [23]:
if not {'timestamp', 'period'}.issubset(tracking_df.index.names):    
    tracking_df['timestamp'] = tracking_df.apply(standardize_timestamp, axis=1)
    tracking_df.set_index(['timestamp', 'half'], inplace=True)
    tracking_df.rename_axis(index={'timestamp': 'timestamp', 'half': 'period'}, inplace=True)
tracking_df.sample(3)

,,match_id,object_id,x,y,z
timestamp,period,,,,,
01:09:05.000000,2,952209,12654,15.53,-16.18,0.0
00:08:50.300000,1,952209,11516,-33.00,12.64,0.0
01:32:56.000000,2,952209,10916,33.16,-4.62,0.0


In [24]:
passes['quantizedTimestamp'] = passes['matchTimestamp'].apply(round_to_tenth_of_second)
if not {'timestamp', 'period'}.issubset(passes.index.names):
    passes.set_index(['quantizedTimestamp', 'matchPeriod'], inplace=True)
    passes.rename_axis(index={'quantizedTimestamp': 'timestamp', 'matchPeriod': 'period'}, inplace=True)

In [25]:
joined_df = passes.join(tracking_df, how='inner', validate='one_to_many')
joined_df.drop(columns=['matchTimestamp'], inplace=True)

In [26]:
columns_to_drop = ['match_id']
existing_columns_to_drop = [col for col in columns_to_drop if col in joined_df.columns]
joined_df.drop(columns=existing_columns_to_drop, inplace=True)

joined_df['object_id'] = joined_df['object_id'].astype(int)

In [27]:
columns_to_prefix = ['object_id', 'x', 'y', 'z']
prefix = 'tracking.'
joined_df.rename(columns={col: prefix + col for col in columns_to_prefix}, inplace=True)

# Normalize Pitch Coordinates

In [28]:
pitch_length = metadata['pitch_length'].values[0]
pitch_width = metadata['pitch_width'].values[0]

In [29]:
start_locations = joined_df.apply(
    lambda row: wyscout_to_pitch(row['location.x'], row['location.y'], pitch_length, pitch_width), 
    axis=1)
end_locations = joined_df.apply(
    lambda row: wyscout_to_pitch(row['pass.endLocation.x'], row['pass.endLocation.y'], pitch_length, pitch_width), 
    axis=1)
joined_df[['location.x', 'location.y']] = start_locations.apply(pd.Series)
joined_df[['pass.endLocation.x', 'pass.endLocation.y']] = end_locations.apply(pd.Series)

# Identify Tracking Objects

In [30]:
def is_self(row):
    return row['player.id.skillcorner'] == row['tracking.object_id']

def is_teammate(row):
    if row['tracking.object_id'] == -1:
        return False
    player_team = player_team_dict[row['player.id.skillcorner']]
    tracking_player_team = player_team_dict[row['tracking.object_id']]
    return player_team == tracking_player_team

In [ ]:
joined_df['is_self'] = joined_df.apply(is_self, axis=1)
joined_df['is_teammate'] = joined_df.apply(is_teammate, axis=1)

In [31]:
joined_df.sample(5)

,,player.id.wyscout,player.id.skillcorner,pass.recipient.id.wyscout,pass.recipient.id.skillcorner,location.x,location.y,pass.endLocation.x,pass.endLocation.y,play_direction,tracking.object_id,tracking.x,tracking.y,tracking.z
timestamp,period,,,,,,,,,,,,,
01:05:08.300000,2,134496,7604,393247,16320,38.750000,-4.227692,36.916667,13.437778,TOP_TO_BOTTOM,10738,-2.81,4.58,0.0
00:13:02.400000,1,535816,33512,370,10738,8.470588,-14.048889,18.000000,-5.636923,TOP_TO_BOTTOM,31930,-4.62,-15.25,0.0
01:22:45.600000,2,116,3837,365446,12654,-24.352941,17.715556,-44.250000,26.715789,BOTTOM_TO_TOP,11630,36.10,4.64,0.0
01:01:47.200000,2,704,10082,476361,14407,-8.470588,28.172632,-1.058824,20.888421,BOTTOM_TO_TOP,946,6.83,-11.51,0.0
00:49:25.300000,2,491458,24466,370,10738,19.058824,4.227692,18.000000,-9.771111,BOTTOM_TO_TOP,14407,-6.58,-22.10,0.0


# Compute ΔxT

In [ ]:
xt_table = pd.read_csv(DATA_PATH + 'smoothed_xt.csv')

In wyscout coordinates are normalized to the range (0,100) in both spatial dimensions.

In [ ]:
cell_width = 100 / xt_table.shape[1]
cell_height = 100 / xt_table.shape[0]

print(f'cell width:\t{round(cell_width,2)}')
print(f'cell height:\t{round(cell_height,2)}')

cell width:	0.95
cell height:	1.47


In [ ]:
def get_xt_index(x, y):
    x_index = min(int(x // cell_width), xt_table.shape[1] - 1)
    y_index = min(int(y // cell_height), xt_table.shape[0] - 1)
    return x_index, y_index

In [ ]:
start_xts = passes.apply(lambda row: xt_table.iat[get_xt_index(row['location.x'], row['location.y'])[1], 
                                                           get_xt_index(row['location.x'], row['location.y'])[0]], axis=1)
end_xts = passes.apply(lambda row: xt_table.iat[get_xt_index(row['pass.endLocation.x'], row['pass.endLocation.y'])[1], 
                                                         get_xt_index(row['pass.endLocation.x'], row['pass.endLocation.y'])[0]], axis=1)
passes.loc[:,'dxt'] = end_xts - start_xts